In [24]:
import pandas as pd
import os

# Define the directory containing the files
directory = '/work/SafeGraph/revelio/code'
fvdirectory = '/work/SafeGraph/revelio/data/yougov'

# Load the YouGov file
df = pd.read_stata(os.path.join(fvdirectory, "ugov_cik.dta"))

# Define a function to check if CIK is valid
def is_valid_cik(cik):
    return pd.notna(cik)

# Filter the DataFrame to include only rows with valid CIKs, 'conm' stands for company name
valid_ciks = df[df['cik'].apply(is_valid_cik)][['cik', 'conm']]

# Sort by 'cik' and 'conm', putting non-empty 'conm' values first
# NaNs are sorted to the end by default
valid_ciks = valid_ciks.sort_values(by=['cik', 'conm'], ascending=[True, False])

# Drop duplicates, keeping the first (which is now the first non-empty 'conm' for each 'cik')
valid_ciks = valid_ciks.drop_duplicates(subset='cik', keep='first')

# Creating the second list (invalid CIKs)
invalid_ciks = df[~df['cik'].apply(is_valid_cik)]['Brand'].drop_duplicates()

valid_ciks.to_csv(os.path.join(directory, 'public_brand_list.csv'), index=False)
invalid_ciks.to_csv(os.path.join(directory, 'private_brand_list.csv'), index=False)
#The remaining code is for testing purposes.

In [9]:
print(df[df['cik']==1800])

         ugov_id      Brand       Date    year  month  week     cik  \
0        22013.0     Ensure 2007-06-03  2007.0    6.0  22.0  1800.0   
1        22017.0   Glucerna 2007-06-03  2007.0    6.0  22.0  1800.0   
2        22019.0  Pedialyte 2007-06-03  2007.0    6.0  22.0  1800.0   
3        22020.0  PediaSure 2007-06-03  2007.0    6.0  22.0  1800.0   
4        22013.0     Ensure 2007-06-06  2007.0    6.0  23.0  1800.0   
...          ...        ...        ...     ...    ...   ...     ...   
18456    22019.0                   NaT  2020.0    8.0   NaN  1800.0   
18457    22013.0                   NaT  2020.0    8.0   NaN  1800.0   
18458    22017.0                   NaT  2020.0    8.0   NaN  1800.0   
18459    22020.0                   NaT  2020.0    8.0   NaN  1800.0   
6099983      NaN                   NaT  2020.0    8.0  35.0  1800.0   

                        conm   gvkey  tik  ... Score_currentcustomer  \
0        ABBOTT LABORATORIES  1078.0  ABT  ...                   NaN   
1  

In [2]:
import pandas as pd
import os
import time
from fuzzywuzzy import fuzz, process

# Function for fuzzy matching
def fuzzy_match(brand, company_list, scorer, cutoff):
    match = process.extractOne(brand, company_list, scorer=scorer, score_cutoff=cutoff)
    return match[0] if match else None

# Paths
directory = '/work/SafeGraph/revelio/code'
sdirectory = '/work/SafeGraph/revelio/data'

# Read the private list
private_list = pd.read_csv(os.path.join(directory, "private_brand_list.csv"))
private_list['Brand'] = private_list['Brand'].str.lower().str.replace(' ', '')

# List to hold dataframes
dfs = []

# Read and process dataframes
for i in range(32):
    file_path = os.path.join(sdirectory, f"company_ref_00{str(i).zfill(2)}_part_00.parquet")
    df = pd.read_parquet(file_path)
    df['company_nospace'] = df['company'].str.lower().str.replace(' ', '')
    df['primary_name_nospace'] = df['primary_name'].str.lower().str.replace(' ', '')
    dfs.append(df)

# Concatenate all dataframes
cik_df = pd.concat(dfs, ignore_index=True)

# Direct Matching
direct_matches = cik_df[cik_df['company_nospace'].isin(private_list['Brand']) | 
                        cik_df['primary_name_nospace'].isin(private_list['Brand'])]

# Add a 'Brand' column to direct_matches
direct_matches = direct_matches.assign(
    Brand=lambda x: x['company_nospace'].where(x['company_nospace'].isin(private_list['Brand']),
                                              x['primary_name_nospace'])
)


# Identifying unmatched entries in private_list
unmatched_brands = private_list[~private_list['Brand'].isin(direct_matches['company_nospace']) &
                                ~private_list['Brand'].isin(direct_matches['primary_name_nospace'])]

# Identifying unmatched entries in cik_df
unmatched_cik_df = cik_df[~cik_df['company_nospace'].isin(direct_matches['company_nospace']) &
                          ~cik_df['primary_name_nospace'].isin(direct_matches['primary_name_nospace'])]

# Preparing company list for fuzzy matching
company_list = unmatched_cik_df['company_nospace'].tolist()

# Perform fuzzy matching for the first observation and measure the time
start_time = time.time()
first_brand = unmatched_brands['Brand'].iloc[0]
first_brand_fuzzy_match = fuzzy_match(first_brand, company_list, scorer=fuzz.WRatio, cutoff=80)
end_time = time.time()

# Print the time taken
print(f"Time taken for fuzzy matching the first observation: {end_time - start_time} seconds")


Time taken for fuzzy matching the first observation: 709.3056681156158 seconds


In [7]:
unmatched_brands.shape


(544, 1)